In [1]:
!pip3 install numpy tifffile --user

  Using cached https://files.pythonhosted.org/packages/87/2d/e4656149cbadd3a8a0369fcd1a9c7d61cc7b87b3903b85389c70c989a696/numpy-1.16.4-cp36-cp36m-manylinux1_x86_64.whl
  Using cached https://files.pythonhosted.org/packages/76/bc/80a4e8dad3f74c3b63aa652dbd2e753b3b110a51d74482d1b245870894eb/tifffile-2019.6.18-py2.py3-none-any.whl


In [2]:
import requests
import json
import numpy as np
import tifffile as tiff
import math
import tempfile
import os
import tiledb
import uuid

from datetime import datetime, timedelta

In [3]:
# tdmq constants
TDMQ_BASE_URL = 'http://web:8000/api/v0.0'
TILEDB_HDFS_ROOT = 'hdfs://hdfs:9000/arrays'
TILEDB_CTX = tiledb.Ctx({'vfs.hdfs.username': 'root'})

In [4]:
# FIXME remember to initialize the db first:
# docker-compose exec web flask db init
def register_thing(thing, description):
    assert isinstance(description, dict)
    # FIXME check if thing already exists and manage errors
    r = requests.post(f'{TDMQ_BASE_URL}/{thing}', json=[description])
    if r.status_code == 500:
        raise ValueError('Illegal value')
    description['code'] = r.json()[0]
    return description


def register_sensor_type(description):
    return register_thing('sensor_types', description)

def register_sensor(description):
    return register_thing('sensors', description)

def register_measure(measure):
    assert isinstance(measure, dict)
    # FIXME check if thing already exists and manage errors
    r = requests.post(f'{TDMQ_BASE_URL}/measures', json=[measure])
    if r.status_code == 500:
        raise ValueError('Illegal value')
    return r.json()

In [5]:
dpc_meteoradar_mosaic_type = {
    "type": "meteoRadar",
    "name": "Mosaic_of_dpc_meteo_radars",
    "brandName": "DPC",
    "modelName": "dpc-radar-mosaic",
    "manufacturerName": "Dipartimento Protezione Civile",
    "category": ["sensor"],
    "function": ["sensing"],
    "controlledProperty": ["VMI", "SRI"], 
    # FIXME no units? Are they defined by the fact that it is a meteoRadar?
    "reference": '/'.join(["http://www.protezionecivile.gov.it",
                           "attivita-rischi/meteo-idro/attivita/previsione-prevenzione",
                           "centro-funzionale-centrale-rischio-meteo-idrogeologico",
                           "monitoraggio-sorveglianza/mappa-radar"]),
}



In [6]:
dpc_temperature_mosaic_type = {
    "type": "temperatureSensorNetwork",
    "name": "Mosaic_of_dpc_temperature_sensors",
    "brandName": "DPC",
    "modelName": "dpc-temperature-mosaic",
    "manufacturerName": "Dipartimento Protezione Civile",
    "category": ["sensor"],
    "function": ["sensing"],
    "controlledProperty": ["TEMP"], 
    # FIXME no units? Celsius
    "reference": '/'.join(["http://www.protezionecivile.gov.it",
                           "attivita-rischi/meteo-idro/attivita/previsione-prevenzione",
                           "centro-funzionale-centrale-rischio-meteo-idrogeologico",
                           "monitoraggio-sorveglianza/mappa-radar"]),
}


In [7]:
dpc_url='http://www.protezionecivile.gov.it/wide-api/wide/product/downloadProduct'

def extract_data(tif):
    page = tif.pages[0]
    data = page.asarray()
    # zeroout nan, force mask value to -9999.0
    data[np.isnan(data)] = -9999.0
    return page.geotiff_tags, data

def gather_data(t, field):
    product_date = math.floor(t.timestamp() * 1000)
    payload = {'productType': field, 'productDate': product_date}
    r = requests.post(dpc_url, json=payload)
    if r.status_code != 200:
        raise ValueError("Bad return code: %s" % r.status_code)
    # FIXME find a cleaner way to handle the returned tif file.
    handle, fpath = tempfile.mkstemp()
    with open(fpath, 'wb') as f:
        f.write(r.content)
    tif = tiff.TiffFile(fpath)
    os.unlink(fpath)
    return extract_data(tif)

def create_sensor_description(sensor_type_desc, name, controlled_properties, time_base, time_delta):
    geotiff_tags, data = gather_data(time_base, controlled_properties[0]) # just to get the data shape
    T = np.array(geotiff_tags['ModelTransformation']).reshape(4, 4)
    ysize, xsize = data.shape
    # Note the nesting level http://wiki.geojson.org/GeoJSON_draft_version_6#Polygon
    coordinates = [[T.dot(v).tolist() for v in [[0, 0, 0, 1], [0, ysize, 0, 1], [xsize, ysize, 0, 1],
                                               [xsize, 0, 0, 1], [0, 0, 0, 1]]]]
    description = {
        "name": name,
        "type": sensor_type_desc['name'],
        # FIXME not used
        "node": "dpi",
        "geometry": {"type": "Polygon", "coordinates": coordinates},
        "controlledProperty": controlled_properties,
        "timebase": time_base.strftime('%Y-%m-%dT%H:%M:%SZ'),
        "timedelta": time_delta.total_seconds(),
        "geotiff_tags": geotiff_tags,
        "grid": {"ysize": ysize, "xsize": xsize},
    }
    return description

In [8]:
try:
    dpc_meteoradar_mosaic_type = register_sensor_type(dpc_meteoradar_mosaic_type)
except ValueError as e:
    print(e)
try:
    dpc_temperature_mosaic_type = register_sensor_type(dpc_temperature_mosaic_type)
except ValueError as e:
    print(e)
res = requests.get(f'{TDMQ_BASE_URL}/sensor_types').json()
sensor_types = dict((d['name'], d) for d in res)
sensor_types

{'Mosaic_of_dpc_meteo_radars': {'brandName': 'DPC',
  'category': ['sensor'],
  'code': 'e6c85a6a-642b-58ed-98f1-f998c9e02717',
  'controlledProperty': ['VMI', 'SRI'],
  'function': ['sensing'],
  'manufacturerName': 'Dipartimento Protezione Civile',
  'modelName': 'dpc-radar-mosaic',
  'name': 'Mosaic_of_dpc_meteo_radars',
  'reference': 'http://www.protezionecivile.gov.it/attivita-rischi/meteo-idro/attivita/previsione-prevenzione/centro-funzionale-centrale-rischio-meteo-idrogeologico/monitoraggio-sorveglianza/mappa-radar',
  'type': 'meteoRadar'},
 'Mosaic_of_dpc_temperature_sensors': {'brandName': 'DPC',
  'category': ['sensor'],
  'code': '3c544888-2af3-55d9-8106-7186f3d51ea9',
  'controlledProperty': ['TEMP'],
  'function': ['sensing'],
  'manufacturerName': 'Dipartimento Protezione Civile',
  'modelName': 'dpc-temperature-mosaic',
  'name': 'Mosaic_of_dpc_temperature_sensors',
  'reference': 'http://www.protezionecivile.gov.it/attivita-rischi/meteo-idro/attivita/previsione-preven

In [9]:
def create_meteo_sensor_tiledb_array(description):
    array_name = os.path.join(TILEDB_HDFS_ROOT, description['code'])
    if tiledb.object_type(array_name) is not None:
        raise ValueError('duplicate object with path %s' % array_name)
    n_rows, n_cols = description['grid']['ysize'], description['grid']['xsize']
    dom = tiledb.Domain(tiledb.Dim(name="delta_t", domain=(0, 12 * 24 * 10 * 365), tile=1, dtype=np.int32),
                        tiledb.Dim(name="rows", domain=(0, n_rows - 1), tile=n_rows, dtype=np.int32),
                        tiledb.Dim(name="cols", domain=(0, n_cols - 1), tile=n_cols, dtype=np.int32),
                        ctx=TILEDB_CTX)
    attrs = [tiledb.Attr(name=aname, dtype=np.float32)
             for aname in description['controlledProperty']]
    schema = tiledb.ArraySchema(domain=dom, sparse=False, attrs=attrs, ctx=TILEDB_CTX)
    # Create the (empty) array on disk.
    tiledb.DenseArray.create(array_name, schema)
    return array_name

In [10]:
!HADOOP_USER_NAME=root hdfs dfs -rm -r hdfs://hdfs:9000/arrays   

rm: `hdfs://hdfs:9000/arrays': No such file or directory


In [11]:
!HADOOP_USER_NAME=root hdfs dfs -mkdir hdfs://hdfs:9000/arrays

In [14]:
# FIXME: time_base should be not earlier that 1 week ago and it should align with the 1 hr boundary
now = datetime.now()
time_base = datetime(now.year, now.month, now.day, now.hour) - timedelta(seconds=6*24*3600)
time_delta_radar = timedelta(seconds=300)
time_delta_temp = timedelta(seconds=3600)

radar_desc = create_sensor_description(dpc_meteoradar_mosaic_type, 'dpc_meteoradar_mosaic', 
                                       ['VMI', 'SRI'], time_base, time_delta_radar)
temp_desc  = create_sensor_description(dpc_temperature_mosaic_type, 'dpc_temperature_mosaic', 
                                       ['TEMP'], time_base, time_delta_temp)

In [15]:
def register_dpc_meteo_sensor(desc):
    try:
        desc = register_sensor(desc)
    except ValueError as e:
        print(e)
    try:
        return create_meteo_sensor_tiledb_array(desc)
    except ValueError as e:
        print(e)

In [16]:
def ingest_dpc_meteo_acquisition(desc, t):
    array_path = os.path.join(TILEDB_HDFS_ROOT, desc['code'])
    assert tiledb.object_type(array_path) is 'array'
    time_base = datetime.strptime(desc['timebase'], '%Y-%m-%dT%H:%M:%SZ')
    i = math.floor((t - time_base).total_seconds() / desc['timedelta'])
    # FIXME: we are assuming that within a sensor all measures use the same mask
    data = {}
    for fname in desc['controlledProperty']:
        print(t, fname, i)
        geotiff_tags, field = gather_data(t, fname)
        data[fname] = field
    with tiledb.DenseArray(array_path, mode='w', ctx=TILEDB_CTX) as A:
       A[i:i+1, :, :] = data
    return register_measure({'time': t.strftime('%Y-%m-%dT%H:%M:%SZ'), 
                             'sensor': desc['name'], 
                             'measure': {'reference': array_path, 'index': i}})

In [17]:
radar_array = register_dpc_meteo_sensor(radar_desc)
temp_array = register_dpc_meteo_sensor(temp_desc)
requests.get(f'{TDMQ_BASE_URL}/sensors').json()

[{'code': '5adee98d-a697-59f1-9bba-c4de0b7846c6',
  'controlledProperty': ['VMI', 'SRI'],
  'geometry': {'coordinates': [[[4.537000517753033,
      47.856095810774605,
      0.0,
      1.0],
     [4.537000517753033, 35.07686201381699, 0.0, 1.0],
     [20.436762466677894, 35.07686201381699, 0.0, 1.0],
     [20.436762466677894, 47.856095810774605, 0.0, 1.0],
     [4.537000517753033, 47.856095810774605, 0.0, 1.0]]],
   'type': 'Polygon'},
  'geotiff_tags': {'GTModelTypeGeoKey': 2,
   'GTRasterTypeGeoKey': 1,
   'GeographicTypeGeoKey': 4326,
   'KeyDirectoryVersion': 1,
   'KeyRevision': 1,
   'KeyRevisionMinor': 2,
   'ModelTransformation': [[0.013249801624104052, 0.0, 0.0, 4.537000517753033],
    [0.0, -0.009128024140684008, 0.0, 47.856095810774605],
    [0.0, 0.0, 0.0, 0.0],
    [0.0, 0.0, 0.0, 1.0]]},
  'grid': {'xsize': 1200, 'ysize': 1400},
  'name': 'dpc_meteoradar_mosaic',
  'node': 'dpi',
  'timebase': '2019-06-22T22:00:00Z',
  'timedelta': 300.0,
  'type': 'Mosaic_of_dpc_meteo_ra

In [18]:
t = time_base
for i in range(100):
    try:
        ingest_dpc_meteo_acquisition(temp_desc, t)
    except ValueError as e:
        print(f'skipping reading at {t} because of {e}.')
    t += time_delta_temp

2019-06-22 22:00:00 TEMP 0
2019-06-22 23:00:00 TEMP 1
2019-06-23 00:00:00 TEMP 2
2019-06-23 01:00:00 TEMP 3
2019-06-23 02:00:00 TEMP 4
2019-06-23 03:00:00 TEMP 5
2019-06-23 04:00:00 TEMP 6
2019-06-23 05:00:00 TEMP 7
2019-06-23 06:00:00 TEMP 8
2019-06-23 07:00:00 TEMP 9
2019-06-23 08:00:00 TEMP 10
2019-06-23 09:00:00 TEMP 11
2019-06-23 10:00:00 TEMP 12
2019-06-23 11:00:00 TEMP 13
2019-06-23 12:00:00 TEMP 14
2019-06-23 13:00:00 TEMP 15
2019-06-23 14:00:00 TEMP 16
2019-06-23 15:00:00 TEMP 17
2019-06-23 16:00:00 TEMP 18
2019-06-23 17:00:00 TEMP 19
2019-06-23 18:00:00 TEMP 20
2019-06-23 19:00:00 TEMP 21
2019-06-23 20:00:00 TEMP 22
2019-06-23 21:00:00 TEMP 23
2019-06-23 22:00:00 TEMP 24
2019-06-23 23:00:00 TEMP 25
2019-06-24 00:00:00 TEMP 26
2019-06-24 01:00:00 TEMP 27
2019-06-24 02:00:00 TEMP 28
2019-06-24 03:00:00 TEMP 29
2019-06-24 04:00:00 TEMP 30
2019-06-24 05:00:00 TEMP 31
2019-06-24 06:00:00 TEMP 32
2019-06-24 07:00:00 TEMP 33
2019-06-24 08:00:00 TEMP 34
2019-06-24 09:00:00 TEMP 35
20

In [19]:
t = time_base
for i in range(100):
    try:
        ingest_dpc_meteo_acquisition(radar_desc, t)
    except ValueError as e:
        print(f'skipping reading at {t} because of {e}.')       
    t += time_delta_radar

2019-06-22 22:00:00 VMI 0
2019-06-22 22:00:00 SRI 0
2019-06-22 22:05:00 VMI 1
2019-06-22 22:05:00 SRI 1
2019-06-22 22:10:00 VMI 2
2019-06-22 22:10:00 SRI 2
2019-06-22 22:15:00 VMI 3
2019-06-22 22:15:00 SRI 3
2019-06-22 22:20:00 VMI 4
2019-06-22 22:20:00 SRI 4
2019-06-22 22:25:00 VMI 5
2019-06-22 22:25:00 SRI 5
2019-06-22 22:30:00 VMI 6
2019-06-22 22:30:00 SRI 6
2019-06-22 22:35:00 VMI 7
2019-06-22 22:35:00 SRI 7
2019-06-22 22:40:00 VMI 8
2019-06-22 22:40:00 SRI 8
2019-06-22 22:45:00 VMI 9
2019-06-22 22:45:00 SRI 9
2019-06-22 22:50:00 VMI 10
2019-06-22 22:50:00 SRI 10
2019-06-22 22:55:00 VMI 11
2019-06-22 22:55:00 SRI 11
2019-06-22 23:00:00 VMI 12
2019-06-22 23:00:00 SRI 12
2019-06-22 23:05:00 VMI 13
2019-06-22 23:05:00 SRI 13
2019-06-22 23:10:00 VMI 14
2019-06-22 23:10:00 SRI 14
2019-06-22 23:15:00 VMI 15
2019-06-22 23:15:00 SRI 15
2019-06-22 23:20:00 VMI 16
2019-06-22 23:20:00 SRI 16
2019-06-22 23:25:00 VMI 17
2019-06-22 23:25:00 SRI 17
2019-06-22 23:30:00 VMI 18
2019-06-22 23:30:00 S